In [11]:
import pandas as pd
import numpy as np

In [2]:
from dgutils.pandas import add_column, add_columns

/Users/alicegao/anaconda2/envs/gk_mz_plot/lib/python2.7/site-packages/genome_kit/data_manager.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
from utils import arr2db, EvalMetric

Using TensorFlow backend.
/Users/alicegao/anaconda2/envs/gk_mz_plot/lib/python2.7/site-packages/scipy/sparse/lil.py:19: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  from . import _csparsetools
/Users/alicegao/anaconda2/envs/gk_mz_plot/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:182: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  from ._shortest_path import shortest_path, floyd_warshall, dijkstra,\
/Users/alicegao/anaconda2/envs/gk_mz_plot/lib/python2.7/site-packages/scipy/sparse/csgraph/_validation.py:5: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  from ._tools import csgraph_to_dense, csgraph_from_dense,\
/Users/alicegao/anaconda2/envs/gk_mz_plot/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:184: Runti

In [4]:
df = pd.read_pickle('result/debug.2020_02_06_1.pkl')

In [5]:
df.head()

,len,one_idx,pred_val,seq,seq_id
0,414,"[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14...","[[0.0, 5.066395e-07, 1.6093254e-06, 1.0728836e...",GUGUGCCCGGCAUGGGUGCAGUCUAUAGGGUGAGAGUCCCGAACUG...,5j01-1-A
1,75,"[[0, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 12,...","[[0.0, 1.013279e-06, 2.9802322e-07, 8.940697e-...",GGCGGCGUAGCUCAGCUGGUCAGAGCGGGGAUCUCAUAAGUCCCAG...,2csx-1-C
2,78,"[[0, 1, 2, 3, 3, 4, 5, 7, 7, 7, 10, 10, 11, 12...","[[0.0, 2.0861626e-07, 1.7881393e-07, 1.8179417...",GGGCCCUUAGCUUAGUCUGGUUAAAGCGAUCGGCUCAUAACCGAUU...,4v8n-1-AW
3,108,"[[0, 1, 2, 3, 4, 5, 6, 6, 7, 8, 9, 10, 11, 12,...","[[0.0, 1.3113022e-06, 1.9073486e-06, 3.5762787...",GAUUUGGGGAGUAGCCGAUUUCCGAAAGGAAAUGUACGUGUCAACA...,4y1m-1-B
4,77,"[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 9, 10, 11, ...","[[0.0, 1.0728836e-06, 4.172325e-07, 8.6426735e...",GGGCCAGGUAGCUCAGUUGGUAGAGCACUGGACUGAAAAUCCAGGU...,4x0b-1-B


In [6]:
df.iloc[1].pred_val.max()

0.038452983

In [7]:
df.iloc[1].pred_val.min()

0.0

In [8]:
df.iloc[1].pred_val.mean()

5.456474e-05

In [10]:
def calculate_metric(one_idx, pred_idx, seq_len, e):
    target = np.zeros((seq_len, seq_len))
    pred = np.zeros((seq_len, seq_len))
    target[one_idx] = 1
    pred[pred_idx] = 1
    sensitivity = e.sensitivity(pred, target)
    ppv = e.ppv(pred, target)
    return sensitivity, ppv, e.f_measure(sensitivity, ppv)

In [27]:
df_metric = []

eval_module = EvalMetric(bypass_pairing_check=True)
# thresholds = [0.0001, 0.0005, 0.001, 0.005, 0.007, 0.01]
thresholds = [0.0001, 0.001, 0.002, 0.003, 0.01]


for _, row in df.iterrows():
    seq_id = row['seq_id']
    seq_len = row['len']
    pred_val = row['pred_val']
    one_idx = row['one_idx']
#     print(seq_id, seq_len)
    for t in thresholds:
        _pred = (pred_val > t).astype(np.int)
        assert np.all((_pred == 1) | (_pred == 0))
        pred_idx = np.where(_pred == 1)
#         print(t)
#         print(calculate_metric(one_idx, pred_idx, seq_len, eval_module))
        metric_s, metric_p, metric_f = calculate_metric(one_idx, pred_idx, seq_len, eval_module)
        df_metric.append({
            'seq_id': seq_id,
            'len': seq_len,
            'threshold': t,
            'sensitivity': metric_s,
            'ppv': metric_p,
            'f_measure': metric_f,
        })

df_metric = pd.DataFrame(df_metric)


/Users/alicegao/anaconda2/envs/gk_mz_plot/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  after removing the cwd from sys.path.


In [28]:
for threshold in thresholds:
    print(threshold)
    print(df_metric[df_metric['threshold'] == threshold][['f_measure', 'sensitivity', 'ppv']].describe())
    print('')

0.0001
        f_measure  sensitivity         ppv
count  254.000000   255.000000  254.000000
mean     0.370117     0.669688    0.299473
std      0.221953     0.157230    0.250346
min      0.044444     0.000000    0.031250
25%      0.219678     0.600000    0.131511
50%      0.281067     0.677419    0.180797
75%      0.469380     0.769231    0.434783
max      1.000000     1.000000    1.000000

0.001
        f_measure  sensitivity         ppv
count  253.000000   255.000000  254.000000
mean     0.554621     0.580502    0.554463
std      0.192018     0.189334    0.239987
min      0.036364     0.000000    0.000000
25%      0.422222     0.479871    0.365385
50%      0.545455     0.589744    0.509865
75%      0.698413     0.707487    0.738171
max      1.000000     1.000000    1.000000

0.002
        f_measure  sensitivity         ppv
count  251.000000   255.000000  254.000000
mean     0.584506     0.516184    0.681916
std      0.199856     0.210456    0.234904
min      0.018868     0.000000   